In [1]:
from igraph import *
import numpy as np
import time
from numba import njit
from scipy.sparse import coo_matrix, dok_matrix, csr_matrix
from collections import defaultdict
import json

def ReloadInflationAs(prefix_string):
    import importlib
    try:
        module=eval(prefix_string)
        globals()[prefix_string]=importlib.reload(module)
    except NameError:
        globals()[prefix_string]=importlib.import_module('InitializeInflation')

In [2]:
TriangleData=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]

inflation_order=2
card=4
TriangleGraph=Graph.Formula("X->A,Y->A:B,Z->B:C,X->C")
EvansGraph=Graph.Formula("U3->A:C:D,U2->B:C:D,U1->A:B,A->C,B->D")
InstrumentalGraph=Graph.Formula("U1->X->A->B,U2->A:B")
#We are going to need to get some code to work with variable ordering.

In [7]:
ReloadInflationAs('it');card=4
obs_count,num_vars,names=it.LearnSomeInflationGraphParameters(TriangleGraph,2)

In [4]:
np.all(it.FindB(TriangleData,2)==it.FindBv2(TriangleData,2))

True

In [5]:
it.FindBv2(TriangleData,2)

array([1.48839896e-02, 5.60451794e-04, 4.26589804e-04, ...,
       4.10999121e-07, 2.59305995e-05, 4.09000869e-04])

In [6]:
"""
start = time.time()
EvansSparseInflationMatrix = inflation.InflationMatrixFromGraph(EvansGraph,2,2)
print('It took', time.time()-start, 'seconds.')
print(EvansSparseInflationMatrix.shape)

start = time.time()
valid_column_orbits = inflation.ValidColumnOrbitsFromGraph(EvansGraph,2,2)
print('It took', time.time()-start, 'seconds.')
print(valid_column_orbits.shape)

start = time.time()
valid_column_orbits = inflation.ValidColumnOrbitsFromGraph(TriangleGraph,2,4)
print('It took', time.time()-start, 'seconds.')
print(valid_column_orbits.shape)
""";

In [6]:
TriangleSparseInflationMatrix = it.InflationMatrixFromGraph(TriangleGraph,2,4)
b=it.FindB(TriangleData,2)
TriangleSparseInflationMatrix.shape

['A', 'B', 'C']


(2080, 2123776)

In [8]:
#TriangleSparseInflationMatrix2 = inflation.optimize_inflation_matrix(TriangleSparseInflationMatrix)

In [8]:
Sol,epsilon=it.InflationLP(TriangleSparseInflationMatrix,b)

Setting up LP in CVXOPT...
Initiating LP
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 6               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 2080
Optimizer  - Cones               

In [11]:
"""So I'm thinking we provide three forms of output. 
**1) a list of coeffients (ideally bolding those which are involved in the inequality)
**2) a raw array of coeffients
**3) a sorted dict of coefficients
**4) a sorted dict of symbols""";

In [53]:
symbolnames=['P('+''.join([''.join(str(i)) for i in idx])+')' for idx in np.ndindex(tuple(np.full(obs_count,card,np.uint8)))]
import sympy as sy
symbols=np.array(sy.symbols(symbolnames))
symb,counts=it.Generate_b_and_counts(symbols, 2)
symbtostring= np.array([str(term).replace('*P','P') for term in symb])

In [65]:
yRaw=np.multiply(np.array(Sol).squeeze(),counts)
y=yRaw.copy()
y[np.abs(y) < np.max(np.abs(y))/1000] = 0;
y2=y/np.unique(np.abs(y))[1];
y3=np.rint(y2).astype(np.int);
inv_counts=(np.lcm.reduce(counts)/counts).astype(np.int)

SolTest=csr_matrix(np.multiply(y3,inv_counts))
np.unique(y3)

array([-2, -1,  0,  1,  2,  3,  4])

In [71]:
csr_matrix(y3)

<1x2080 sparse matrix of type '<class 'numpy.intc'>'
	with 430 stored elements in Compressed Sparse Row format>

In [72]:
"Validation:";
check=(SolTest * TriangleSparseInflationMatrix)
check.min()

In [78]:
#encoding=it.GenerateEncodingMonomialToRow(card**obs_count,inflation_order)
#s,idx,counts=np.unique(encoding,return_index=True,return_counts=True)
#yprime=np.multiply(counts,y3.squeeze())
#yprime=(yprime/np.gcd.reduce(yprime)).astype(np.int)


indextally=defaultdict(list)
[indextally[str(val)].append(i) for i,val in enumerate(y3) if val != 0]
#for coefficient in np.unique(yprime):
#    print({coefficient:np.array(tally[coefficient])})
#for i, vals in tally.items():
#        tally[i] = np.array(vals)
#tally.items()
indextally.items()

dict_items([('2', [3, 7, 19, 23, 190, 194, 198, 202, 206, 210, 214, 218, 222, 226, 230, 234, 238, 242, 246, 253, 257, 265, 269, 273, 281, 285, 289, 428, 432, 436, 440, 444, 448, 452, 456, 460, 464, 468, 472, 476, 480, 487, 491, 495, 499, 503, 507, 511, 519, 524, 525, 526, 527, 579, 580, 581, 582, 633, 634, 635, 636, 650, 654, 658, 662, 666, 670, 674, 678, 682, 686, 687, 688, 689, 690, 694, 698, 705, 709, 713, 717, 721, 722, 723, 724, 725, 729, 733, 741, 742, 743, 744, 745, 773, 774, 775, 776, 793, 794, 796, 823, 824, 825, 826, 843, 844, 845, 846, 856, 864, 868, 872, 873, 874, 875, 876, 880, 884, 888, 892, 893, 894, 895, 896, 900, 907, 911, 915, 919, 923, 927, 931, 939, 943, 951, 1046, 1050, 1054, 1058, 1062, 1066, 1070, 1074, 1078, 1082, 1086, 1093, 1097, 1121, 1125, 1220, 1224, 1228, 1232, 1236, 1240, 1244, 1248, 1252, 1256, 1263, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1279, 1284, 1285, 1286, 1287, 1291, 1295, 1299, 1307, 1308, 1310, 1311, 1312, 1313, 1314, 1323, 1324, 1325, 

In [57]:
symboltally=defaultdict(list)
for i, vals in indextally.items():
    symboltally[i] = symbtostring[vals].tolist() 
symboltally.items();
#print(json.dumps(coefftally, separators=(',', ':')))

In [64]:
#Note: symbolic ADDITION is very slow, so we do NOT do: Inequality_as_string='0<='+str(np.dot(y3,symb)).replace('*P','P')
final_ineq_WITH_ZEROS=np.multiply(y3,symb)
final_ineq_WITHOUT_ZEROS=final_ineq_WITH_ZEROS[y3!=0]
Inequality_as_string='0≤'+"+".join([str(term) for term in final_ineq_WITHOUT_ZEROS]).replace('*P','P')
Inequality_as_string=Inequality_as_string.replace('+-','-')

In [61]:
returntouser={
 'Raw Solver Output': yRaw.tolist(),
 'Inequality_as_string': Inequality_as_string,
 'Coefficients_grouped_by_index': indextally,
 'Coefficients_grouped_by_symbol': symboltally,
# 'b_vector_position': idx.tolist(),
 'Clean_Solver_Output': y3.tolist(),
 'Symolic_Association': symbtostring.tolist()
};
f = open('inequality_output.json', 'w');
import json
print(json.dumps(returntouser), file=f)
f.close()
